<a href="https://colab.research.google.com/github/Gogulaanand/Fault-Diagnosis/blob/master/Fault%20Diagnosis_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import io
import tensorflow as tf


In [154]:
from google.colab import files
uploaded = files.upload()

Saving projdata_test_set.csv to projdata_test_set (5).csv
Saving projdata.csv to projdata (6).csv


In [0]:
df=pd.read_csv(io.StringIO(uploaded['projdata.csv'].decode("utf-8")),engine='python',sep=",")
df1=pd.read_csv(io.StringIO(uploaded['projdata_test_set.csv'].decode("utf-8")),engine='python',sep=",")

ds=df.sample(frac=1)
ds1=df1.sample(frac=1)


In [0]:
Y_train=np.array(df[['Fault']]).T
X_train=ds.as_matrix(columns=df.columns[1:]).T
X_test=ds1.as_matrix(columns=df1.columns[1:]).T
Y_test=np.array(df1[['Fault']]).T
a=X_train
b=Y_train
c=X_test
d=Y_test
X_train=tf.constant(X_train,name="X_train",dtype=tf.float64)
Y_train=tf.constant(Y_train,name="Y_train",dtype=tf.float64)
X_test=tf.constant(X_test,name="X_test",dtype=tf.float64)
Y_test=tf.constant(Y_test,name="Y_test",dtype=tf.int64)


In [157]:
nx=np.min(np.shape(X_train))
ny=np.min(np.shape(Y_train))
nh=3
nl=2
print("Size of input layer: "+str(nx))
print("Size of output layer: "+str(ny))
print("Size of hidden units in each layer: "+str(nh))
print("No of hidden layers: "+str(nl))


Size of input layer: 3
Size of output layer: 1
Size of hidden units in each layer: 3
No of hidden layers: 2


In [0]:
def sigmoid(z):
  
  x=tf.placeholder(tf.float64,name="x")
  
  sigmoid=tf.sigmoid(x)
  
  with tf.Session() as sess:
    result=sess.run(sigmoid,feed_dict={x:z})
  sess.close()
  
  return result

In [0]:
def create_placeholder(nX,nY):
  
  X=tf.placeholder(tf.float64,shape=(nX,None))
  Y=tf.placeholder(tf.float64,shape=(1,None))
  
  return X,Y

In [0]:
def initialize_parameters():
  
  with tf.variable_scope("initialize_parameters",reuse=tf.AUTO_REUSE):
    
  
    W1 = tf.get_variable("W1",[3,3],dtype=tf.float64,initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable("b1",[3,1],dtype=tf.float64,initializer=tf.zeros_initializer())
    W2 = tf.get_variable("W2",[3,3],dtype=tf.float64,initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable("b2",[3,1],dtype=tf.float64,initializer=tf.zeros_initializer())
    W3 = tf.get_variable("W3",[1,3],dtype=tf.float64,initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable("b3",[1,1],dtype=tf.float64,initializer=tf.zeros_initializer())
    

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [0]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    
    Z1 = tf.add(tf.matmul(W1,X),b1)                                              # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                              # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2,A1),b2)                                              # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                              # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3,A2),b3)                                              # Z3 = np.dot(W3,Z2) + b3
    
    return Z3
    

In [0]:
def compute_cost(Z3, Y):
    
    
    
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=labels))
    
    
    return cost

In [0]:
def random_mini_batches(X,Y,mini_batch_size=128):
  
  m=y.shape[1]
  
  
  #permutation=list(np.random.permutation(int(m)))      #to shuffle the x and y matrices
  #print(permutation)
  #shuffled_X=X[:,permutation]
  #shuffled_Y=y[:,permutation].reshape((1,m))
  shuffled_X=X
  shuffled_Y=y

  mini_batches=[]
  
  total_possible_mini_batches=int(math.floor(int(m)/int(mini_batch_size)))
  
  for k in range(total_possible_mini_batches):
    
    mini_batch_X=shuffled_X[:,(mini_batch_size*k):(mini_batch_size*(k+1))]
    mini_batch_Y=shuffled_Y[:,(mini_batch_size*k):(mini_batch_size*(k+1))]
    
    mini_batch=(mini_batch_X,mini_batch_Y)
   
    mini_batches.append(mini_batch)
    
  
  if m%mini_batch_size!=0:
    
    mini_batch_X=shuffled_X[:,total_possible_mini_batches*mini_batch_size:m]
    mini_batch_Y=shuffled_Y[:,total_possible_mini_batches*mini_batch_size:m]
   
    mini_batch=(mini_batch_X,mini_batch_Y)
    mini_batches.append(mini_batch)
  
    
  return mini_batches

In [0]:
def model(X_train,Y_train,X_test,Y_test, learning_rate = 0.005,num_epochs = 30, minibatch_size = 128, print_cost = True):
  
    
    
    with tf.variable_scope("model",reuse=tf.AUTO_REUSE):              
      costs = []                                       

      (nX,m)=X_train.shape
      nY=Y_train.shape[0]
      X,Y=create_placeholder(nX,nY)
      
      parameters = initialize_parameters()

      Z3 = forward_propagation(X,parameters)

      cost = compute_cost(Z3,Y)

      optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

      init = tf.global_variables_initializer()

      with tf.Session() as sess:

          sess.run(init)

          for epoch in range(num_epochs):

              epoch_cost = 0                       
              num_minibatches = int(m//minibatch_size) 
              minibatches = random_mini_batches(X_train,Y_train,minibatch_size)

              for minibatch in minibatches:




                  (minibatch_X, minibatch_Y) = minibatch
                  
                  #x=tf.placeholder(tf.float64,shape=minibatch_X.shape,name="x")
                  #y1=tf.placeholder(tf.float64,shape=minibatch_Y.shape,name="y")


                  _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X.eval(), Y: minibatch_Y.eval()})


                  epoch_cost += int(minibatch_cost) / int(num_minibatches)


              if print_cost == True:
                  print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
                  costs.append(epoch_cost)


          plt.plot(np.squeeze(costs))
          plt.ylabel('cost')
          plt.xlabel('iterations (per tens)')
          plt.title("Learning rate =" + str(learning_rate))
          plt.show()


          trained_parameters = sess.run(parameters)
   
          correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y_train))
          accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
          print ("Train Accuracy:", accuracy.eval({X:a, Y:b}))
        
          
          z3=forward_propagation(X_test,trained_parameters)
          correct_prediction = tf.equal(tf.argmax(z3), tf.argmax(Y_test))
          accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
          print ("Test Accuracy:", accuracy.eval({X:c, Y:d}))
          
          #acc_train=tf.metrics.accuracy(Z3,b)
          #print(acc_train)
          return trained_parameters
        

In [0]:
trained_parameters = model(X_train,Y_train,X_test,Y_test)

Cost after epoch 0: 0.291667
Cost after epoch 1: 0.291667
Cost after epoch 2: 0.291667


In [0]:
z3=forward_propagation(X_test,trained_parameters)
A3=tf.round(tf.math.sigmoid(z3))
#p=tf.argmax(Y_test,1)
#p=[1 if x>0.5 else 0 for x in A3]
#predict=tf.argmax(A3,axis=1,output_type=tf.int32,keep_dims=True)
#correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y_test))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#print ("Test Accuracy:", accuracy.eval({X:c, Y:d}))
#acc=tf.metrics.accuracy(Y_test,p,weights=0)
#print("Test Accuracy: ",acc)

  
  

In [171]:
print(X_test.shape)
print(A3.shape)
print(Y_test.shape)
print(p.shape)

with tf.Session() as sess:
  sess.run(init)
  print(sess.run(tf.count_nonzero(A3)))
  #c=tf.map_fn(, A3)




(3, 125)
(1, 125)
(1, 125)
(1,)
[[1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0.
  0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0.
  1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1.
  1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1.
  0. 0. 1. 1. 1.]]
84
